# Forecasting con Amazon Forecast

## Introducción
En este notebook se explica cómo utilizar la librería **sibila**, que permite interactuar con el servicio de Amazon Forecast utilizando Python.

Para continuar, será necesario instalar la lib **sibila** disponible en nuestro pypi.


In [1]:
!python3 -m pip install --index-url http://pypi.ml.com/simple/ \
    --trusted-host pypi.ml.com \
    sibila

Looking in indexes: http://pypi.ml.com/simple/
You should consider upgrading via the '/Users/luan2/lab/fury_python-bi-automl/.venvSIB/bin/python3 -m pip install --upgrade pip' command.



## Setup del proyecto

Una vez instalada, instanciamos un **Project**, que es la entidad principal a partir de la cual vamos a poder subir **Datasets** y crear **Forecasts**.

In [1]:
from sibila import Project, Dataset, Predictor


Las credenciales de AWS que se necesitan para poder interactuar con el servicio se especifican de la siguiente manera:

_Nota: Si estamos trabajando en un entorno con Bastion, no es necesario pasarle al **Project** las credenciales de AWS, ya que éste las levanta automáticamente del entorno._

In [2]:
mi_proyecto = Project(
    name="point_example",
    team="bi-ml-cross",
    username="ldap_user",
    password="ldap_pass",
    #aws_access_key_id='accessKey',
    #aws_secret_access_key='secretKey',
    #aws_session_token='sessionToken',
    #region_name="us-east-1",
)

## Creación del dataset

Para poder hacer una predicción, primero necesitamos entrenar un predictor a partir de un dataset.
Como dataset principal, amazon requiere un dataset del tipo **TARGET TIME SERIES**. También podemos crear datasets del tipo **RELATED TIME SERIES** y un **ITEM METADATA** para agregar más información.

Para cada tipo de dataset, amazon también nos requiere especificar un ***schema*** mediante el cual le comunicamos al servicio qué información le proporcionamos en cada columna del dataset.

In [8]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      }
   ]
}

In [9]:
# Creacion del dataset.
dataset_entrenamiento = mi_proyecto.upload_dataset(
    local_path='./data/point.csv',
    ds_type=Dataset.TARGET_TIME_SERIES,
    schema=schema,
    frequency='D',
    timestamp_format='yyyy-MM-dd',
)

INFO:sibila:Uploading dataset to s3


## Entrenamiento de un predictor

Ahora que nuestro proyecto contiene al menos el **TARGET TIME SERIES** dataset, podemos crear un predictor para nuestro proyecto.

El predictor se entrenará con los datasets que contenga el proyecto al momento de la creación.

In [ ]:
mi_predictor = mi_proyecto.train_new_predictor(
    name='predictor_point',
    algorithm=Predictor.AUTO,
    frequency='D',
    horizon=30,
)


Los algoritmos de entrenamiento que podemos utilizar son: 'ARIMA', 'Deep_AR_Plus', 'ETS', 'NPTS' y 'Prophet'. También podemos especificar 'AUTO' y Amazon elegirá el mejor predictor luego de haber entrenado y comparado con los distintos algoritmos.

La frecuencia representa la frecuencia de predicción a futuro que podrá hacer nuestro predictor, es decir: con una frecuencia diaria, nuestro predictor podrá decirnos qué demanda de item tendremos por cada día futuro. (la frecuencia especificada no puede ser mayor que la del **TARGET TIME SERIES** que hayamos subido a nuestro proyecto)

El horizonte representa qué tan a futuro (en unidades de la frecuencia especificada) puede predecir nuestro predictor respecto al último dato que haya en nuestro dataset al momento de realizar la predicción. Por ejemplo: si nuestra frecuencia es 'D', nuestro horizonte es 30, y subimos un **TARGET TIME SERIES** que termina el 20 de Abril, podremos predecir la demanda hasta el 30 de Mayo.

### Métricas del entrenamiento

Usando el método *metrics()* de un predictor, podemos exportar las métricas de entrenamiento del predictor.
Como parámetro debemos brindar únicamente una ruta local al directorio donde quedarán guardados los dos archivos csv que provee el servicio:
- **backtest_forecasts.csv**; donde encontramos las comparaciones entre las predicciones y los valores reales para los períodos de backtesting.
- **backtest_metrics.csv**; donde encontramos métricas cualitativas como por ejemplo el RMSE.


In [ ]:
mi_predictor.metrics('./data/metrics')

### Generación de predicciones

Teniendo un predictor entrenado, podemos hacer una predicción y exportar los resultados de la siguiente forma:

In [ ]:
# Crear una prediccion y exportarla a resultados.csv
mi_predictor.forecast('./resultados.csv')

Ésto genera un archivo local con los resultados del forecast.

## Recuperar un predictor

Sibila nos permite ver qué predictores tenemos creados para nuestro proyecto de la siguiente manera:

In [17]:
mi_proyecto.predictors()

['predictor_point']

Podemos recuperar cualquiera de los predictores que figuren en la lista para hacer predicciones o ver sus métricas:

In [8]:
predictor_arima = mi_proyecto.get_predictor('predictor_point')
predictor_arima.metrics('./metricas_ARIMA')